In [1]:
import os
import requests
import gradio as gr
from dotenv import load_dotenv
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

# ----- Load or set your API keys -----
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")  # Your key provided

# ----- Gemini Setup -----
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    google_api_key=GOOGLE_API_KEY,
    convert_system_message_to_human=True
)

# ----- Web Search Tool -----
search_tool = DuckDuckGoSearchRun()

# ----- Weather Tool -----
@tool
def get_weather_forecast(city: str) -> str:
    """
    Provides current and forecast weather info for a city.
    """
    try:
        response = requests.get(
            "http://api.weatherapi.com/v1/forecast.json",
            params={"key": WEATHER_API_KEY, "q": city, "days": 3, "aqi": "no", "alerts": "no"}
        )
        response.raise_for_status()
        data = response.json()
        current = data["current"]
        location = data["location"]
        forecast = data["forecast"]["forecastday"]

        message = f"🌤️ Weather in {location['name']}, {location['country']}:\n\n"
        message += f"📍 Temperature: {current['temp_c']}°C | Feels Like: {current['feelslike_c']}°C\n"
        message += f"💧 Humidity: {current['humidity']}% | 💨 Wind: {current['wind_kph']} km/h {current['wind_dir']}\n"
        message += "\n📅 3-Day Forecast:\n"
        for day in forecast:
            message += f"- {day['date']}: {day['day']['condition']['text']}, "
            message += f"🌡️ {day['day']['maxtemp_c']}°C / {day['day']['mintemp_c']}°C, "
            message += f"☔ Rain Chance: {day['day']['daily_chance_of_rain']}%\n"
        return message

    except Exception as e:
        return f"❌ Weather fetch failed: {str(e)}"

# ----- Attractions Tool -----
@tool
def get_tourist_attractions(city: str) -> str:
    """
    Returns popular places to visit in a city using DuckDuckGo.
    """
    try:
        query = f"Top tourist attractions in {city}"
        result = search_tool.run(query)
        return f"🏛️ Attractions in {city}:\n\n{result.strip()}\n\n✨ Travel Tips:\n• Check opening times\n• Use public transport\n• Try local dishes!"
    except Exception as e:
        return f"❌ Attraction search failed: {str(e)}"

# ----- LangChain Agent Setup -----
tools = [get_weather_forecast, get_tourist_attractions]

prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a friendly travel assistant 🌍

Provide:
- 🌤️ Weather info
- 🏛️ Tourist attractions
- ✨ Travel tips

Use tools to answer accurately. Use emojis and organize info clearly.
"""),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

# ----- Travel Assistant Logic -----
def travel_assistant(city: str) -> str:
    if not city.strip():
        return "⚠️ Please enter a city name."
    try:
        result = agent_executor.invoke({"input": city})
        return result.get("output", "❌ Assistant did not return a response.")
    except Exception as e:
        return f"❌ Error processing your request: {str(e)}"

# ----- Gradio UI -----
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## ✈️ Travel Assistant\nPlan your trip with live weather & top attractions!")
    city_input = gr.Textbox(label="Enter City", placeholder="e.g., Rome, Tokyo, New York")
    submit_btn = gr.Button("Explore")
    result_box = gr.Textbox(label="Travel Information", lines=20, interactive=False)
    submit_btn.click(fn=travel_assistant, inputs=city_input, outputs=result_box)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://66516c1f16d93a9cc8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
